# Goal

We want to migrate our machines and apps to a new system. To do so we consider having:
- $N$ machines, each having different CPU/Memory abilities.
- $A$ apps, which have distinct CPU/memory requirements and can run in any subset of the $N$ machines
- A source and target cluster manager. At first All physical machines are running in the source, and we want the all to run in the target cluster.

In [72]:
from dataclasses import dataclass
from typing import List, Set

@dataclass(frozen=True)
class Machine:
    name: str
    cpu: int
    memory: int
    migrated: bool = False
    
    @property 
    def id(self):
        return self.__hash__()

@dataclass(frozen=True)
class App:
    name: str
    cpu: int
    memory: int
    machines: Set[Machine]
    
    @property 
    def id(self):
        return self.__hash__()

# State

We define the state of our system as the list of machines and associated apps. We also provide a "slack" function that approximates the 

In [73]:
@dataclass(frozen=True)
class State:
    machines: Set[Machine]
    apps: Set[App]
    
    @property 
    def id(self):
        return self.__hash__()
    
    # This method is good enough for finding solutions, but better slack functions exist
    def minimum_slack_for_an_app(self) -> float:
        minimum_cpu_slack = float("inf")
        minimum_memory_slack = float("inf")
        
        for app in self.apps:
            
            cpu_slack = 0
            memory_slack = 0
            
            for machine in app.machines:
                cpu_needed = sum([app.cpu for app in self.apps if machine in app.machines])
                cpu_slack += machine.cpu - cpu_needed
                memory_needed = sum([app.memory for app in self.apps if machine in app.machines])
                memory_slack += machine.cpu - memory_needed

            minimum_cpu_slack = min(minimum_cpu_slack, cpu_slack)
            minimum_memory_slack = min(minimum_memory_slack, memory_slack)
            
        return min(minimum_cpu_slack, minimum_memory_slack)


    def n_machines_to_migrate(self) -> int:
        return len([m for m in self.machines if not m.migrated])
    
    def least_number_of_apps_in_non_migrated_machine(self) -> float:
        assert len([machine for machine in self.machines if machine.migrated==False]), "All machines have been migrated"
        return min([
            len([app for app in self.apps if machine in app.machines])
            for machine
            in self.machines
            if machine.migrated==False
        ])

In [74]:
machines = [
    Machine(name="M1", cpu=5, memory=6, migrated=False),
    Machine(name="M2", cpu=3, memory=4, migrated=False),
    Machine(name="M3", cpu=2, memory=2, migrated=False),
]
apps = [
    App(name="A1", cpu=1, memory=1, machines=frozenset(machines[0:2])),
    App(name="A2", cpu=1, memory=1, machines=frozenset(machines[2:3])),
]
state = State(machines=frozenset(machines), apps=frozenset(apps))

# A* algorithm

Given that we have structured our states, we can apply the A* algorithm. We require a
- generate_neighbours function, which sets both the path cost $f$ and remaining estimated cost $h$
- boolean method to determine whether a state is a target 

In [ ]:
from abc import ABC, abstractmethod
from typing import Generic, TypeVar, Dict, Optional
from dataclasses import field

T = TypeVar("T")

@dataclass
class Path:
    head: T
    previous_state: Optional[Path]
    message: Optional[str] = None
    path: List[T] = field(default_factory=lambda: [])
    f: float = 0
    h: float = float("inf")

class MigrateClusters:
    # https://en.wikipedia.org/wiki/A*_search_algorithm

    def __init__(self, initial_configuration: T):
        self.paths_to_explore: Set[Path] = {Path(head=initial_configuration)}
        self.explored_states: Set[T] = set()
    
    @abstractmethod
    def neighbours(self, state: Path) -> Set[Path]:
        pass
    
    @abstractmethod
    def is_target_state(self, state: Path) -> bool:
        pass
    
    # This contains the core of the A* algorithm
    def get_result(self):
        
        is_target_reached = len([
            partial_path
            for partial_path
            in self.paths_to_explore
            if self.is_target_state(partial_path.head)
        ]) > 0 
        if is_target_reached:
            return [state for state in self.paths_to_explore.keys() if self.is_target_state(state)][0]
        
        current_path = min(self.paths_to_explore, key=lambda pp: pp.f + pp.h)
        new_neighbours = [
            n
            for n in self.neighbours(current_path)
            if n.head not in 
               [p.head for p in self.explored_states + self.paths_to_explore]
        ] # TODO: I should keep the minimum cost here rather than the 1st found solution :/
        self.paths_to_explore |= new_neighbours 
        

In [22]:
z = {1,2}

In [31]:
z |= {6,4}
z

{1, 2, 3, 4, 6}

In [30]:
{1,2} | {5,4}

{1, 2, 4, 5}